In [54]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import os
import matplotlib.pyplot as plt

In [2]:
#xl = pd.ExcelFile('Retiros 2015.xlsx')

#xl.sheet_names  # see all sheet names
File_Name = 'Retiros 2016.xlsx'

wb = load_workbook(filename = File_Name, use_iterators = True)
List_Sheets = wb.get_sheet_names()

C:\Anaconda3\lib\site-packages\openpyxl\workbook\names\named_range.py:124: UserWarning: Discarded range with reserved name
  warnings.warn("Discarded range with reserved name")


In [4]:
frame = pd.DataFrame()
list_ = []

for Month in List_Sheets:  
    df = pd.read_excel(File_Name,sheetname= Month, skiprows=0, header = None)
    
    list1 = df.iloc[0,:].tolist()
    list2 = df.iloc[1,:].tolist()
    list3 = df.iloc[2,:].tolist()
    A = ['{}*{}*{}'.format(a,b,c) for a, b,c in zip(list1, list2,list3)]
    A = [elem.strip() for elem in A]
    A[0] = 'Date'
    A[1] = 'Hour'

    df.columns = A
    df = df.iloc[4:,:]

    df = df.set_index(['Date','Hour'])
    df = df.groupby(df.columns, axis=1).sum()
    
    list_.append(df)
    
frame = pd.concat(list_,axis = 0)

In [6]:
frame = frame.loc[:,(frame.sum(axis=0) != 0)]

In [101]:
aux = frame.reset_index()
aux['Time'] = pd.to_datetime(aux['Date']+pd.to_timedelta((aux['Hour']-1),unit='h') )
aux = aux.drop(['Date','Hour'],1)
aux = aux.set_index(['Time'])

In [102]:
frame2 = aux.transpose()
frame2 = frame2.reset_index()

frame2['Bus'], frame2['Cliente'], frame2['Gen']  = frame2['index'].str.split('*', 2).str
frame2 = frame2.drop(['index'],1)

Dates = frame2.columns
#frame2.columns = [i for i in range(frame2.shape[1]-3)] +['Bus','Cliente','Gen']
#frame2 = frame2.apply(pd.to_numeric, errors='ignore')

frame4 = frame2.groupby(['Cliente','Bus'],as_index=False).sum()
frame4 = frame4.set_index(['Cliente','Bus']).transpose()

In [103]:
frame4.columns = ['%s%s' % (a, '*%s' % b if b else '') for a, b in frame4.columns]

In [70]:
frame4['ATACAMA MINERALS | Aguas Blancas 13 kV'].plot().set_title('ATACAMA MINERALS | Aguas Blancas 13 kV')
plt.show()

In [66]:
a.savefig('temp.png', dpi=fig.dpi)

AttributeError: 'NoneType' object has no attribute 'savefig'

In [99]:
plt.clf()
i = 0
for column in frame4:
    ax = frame4[column].plot().set_title(str(i)+'_'+column)
    plt.ylabel('Retiro Horario[MWh]')
    fig = ax.get_figure()
    if i <10:
        path = "01 - Graficos Retiro/00"+ str(i) +'_' +column+ '.png'
    elif i <100:
        path = "01 - Graficos Retiro/0"+ str(i) +'_' +column+ '.png'
    else:
        path = "01 - Graficos Retiro/"+ str(i) +'_' +column+ '.png'
        
    fig.savefig(path, dpi=fig.dpi)
    plt.clf()
    i = i+1
    del ax, fig

In [88]:
frame4['ATACAMA MINERALS_Aguas Blancas 13 kV'].plot().set_title('ATACAMA MINERALS_Aguas Blancas 13 kV')
fig = plt.figure()

C:\Anaconda3\lib\site-packages\matplotlib\pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [124]:
writer = pd.ExcelWriter('Retiro3_2016.xlsx')
frame3.to_excel(writer,'Cliente-Gen',startrow=2,index=True,header=True)
frame4.to_excel(writer,'Cliente',startrow=2,index=True,header=True)
writer.save()

In [104]:
writer = pd.ExcelWriter('Retiro5_2016.xlsx')
frame4.to_excel(writer,'Cliente',startrow=2,index=True,header=True)
writer.save()